# Threshold Signature Scheme

In [1]:
from typing import List, Tuple


def fit(points: List[Tuple[int, int]], m: int):
    n = len(points)
    coefficients = [0] * n

    def basis_polynomial(i: int) -> List[int]:
        xi, _ = points[i]
        basis = [1]

        for j, (xj, _) in enumerate(points):
            if i != j:
                # Polynomial multiplication (basis * (x - xj))
                new_basis: List[int] = []
                for k in range(len(basis) + 1):
                    if k == 0:
                        new_basis.append(-xj * basis[k] % m)
                    elif k == len(basis):
                        new_basis.append(basis[k - 1])
                    else:
                        new_basis.append((basis[k - 1] - xj * basis[k]) % m)
                basis = new_basis

                # Divide by (xi - xj) mod m
                inv = pow(xi - xj, -1, m)
                for k in range(len(basis)):
                    basis[k] = basis[k] * inv % m

        return basis

    for i, (_, yi) in enumerate(points):
        li = basis_polynomial(i)
        for j in range(len(li)):
            coefficients[j] = (coefficients[j] + yi * li[j]) % m

    def f(x: int) -> int:
        result = 0
        power_of_x = 1  # x^0 initially
        for coefficient in coefficients:
            result = (result + coefficient * power_of_x) % m
            power_of_x = (power_of_x * x) % m
        return result

    return f

In [2]:
from abc import ABC, abstractmethod
import hashlib
import random
from typing import Any

# TODO: E on ed25519
# from ecpy.curves import Curve, Point
# curve = Curve.get_curve('Ed25519')


def i2b(x: int):
    return x.to_bytes((x.bit_length() + 7) // 8)


class E(ABC):
    @abstractmethod
    def __add__(self, other: Any) -> "E":
        raise NotImplementedError()

    @abstractmethod
    def __mul__(self, other: int) -> "E":
        raise NotImplementedError()

    @abstractmethod
    def __eq__(self, other: Any) -> bool:
        raise NotImplementedError()
    
    @abstractmethod
    def data(self) -> bytes:
        pass

    @property
    @abstractmethod
    def order(self) -> int:
        pass


class MockE(E):
    M: int = pow(2, 31) - 1

    def __init__(self, x: int) -> None:
        self._n = x % self.M
    
    def __add__(self, other: Any) -> "E":
        if not isinstance(other, MockE):
            raise TypeError()
        return MockE(other._n + self._n)

    def __mul__(self, other: int) -> "E":
        return MockE(self._n*other)

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, MockE):
            return False
        return other._n == self._n
    
    def data(self) -> bytes:
        return i2b(self._n)

    def __repr__(self) -> str:
        return str(self._n)
    
    @property
    def order(self):
        return self.M


def message_hash(R: E, public_k: E, message: bytes):
    assert R.order == public_k.order
    hasher = hashlib.sha256()
    hasher.update(R.data())
    hasher.update(public_k.data())
    hasher.update(message)
    return int.from_bytes(hasher.digest(), byteorder='big') % R.order


class Peer:
    def __init__(self, i: int, private_key: int, group_public_key: E, generator: E):
        assert generator.order == group_public_key.order
        self._generator = generator
        self._i = i
        self._private_key = private_key
        self._salt = i2b(random.randint(1, generator.order))
        self.group_public_key = group_public_key
    
    def _r(self, entropy: bytes):
        h = hashlib.sha256()
        h.update(self._salt)
        h.update(entropy)
        return int.from_bytes(h.digest(), byteorder='big') % self._generator.order
    
    def _message_entropy(self, message: bytes):
        h = hashlib.sha256()
        h.update(i2b(self._i))
        h.update(self.group_public_key.data())
        h.update(message)
        return h.digest()
    
    def phase1(self, entropy: bytes):
        R = self._generator * self._r(entropy)
        return self._i, R
    
    def phase2(self, R_sg: E, message: bytes):
        r = self._r(self._message_entropy(message))
        h = message_hash(R_sg, self.group_public_key, message)
        return (r + h * self._private_key) % self._generator.order

## Generating group privates

This needs to be done very secretly since this is the only moment when the private key for the whole group can be calculated

1. Generate a polynomial `f(index) = PrivateKey` from `N` random key points `f = fit([rand, rand for _ in range(N)])`
2. Calculate M (M >= N) private keys like this `PeersPrivateKeys = [f(i+1) for i in range(M)]`
3. Calculate the public key of the group account `GroupPubKey = E(f(0))`
4. Put private keys to peers ensuring secure channel
5. Put group public key to peers ensuring no man in middle
6. Each peer then inside generates a random `Salt` (that does not change from the moment of generation)

In [3]:
def generate_random_points(n: int, order: int):
    points: List[Tuple[int, int]] = []
    for _ in range(n):
        x = random.randint(1, order-1)
        y = random.randint(1, order-1)
        points.append((x, y))
    return points

generator = MockE(1)

N, M = 3, 5
random_points = generate_random_points(N, generator.order)
privates_keys_polynomial = fit(random_points, generator.order)

def x_i(i: int):
    return i + 1

def lx(i: int, x: int):
    assert i >= 0

    _x_i = x_i(i)
    res = 0
    for j in range(M):
        if j == i:
            continue
        _x_j = x_i(j)
        term = (x - _x_j) * pow(_x_i - _x_j, -1, generator.order) % generator.order
        res = (res + term) % generator.order
    
    return res

group_public_key = generator * privates_keys_polynomial(0)

peers = [Peer(i, privates_keys_polynomial(x_i(i)), group_public_key, generator) for i in range(M)]

## Signing

### Phase 1:

Dealer:
1. Select `N` peers (subset from `M`)
1. Send each of them `Entropy = hash(Message)`

Peer:
1. Generate `r = hash(Salt, Entropy)`
1. Send `R = E(r)` and `x`

Dealer:
1. Collect `R`'s from `N` peers, for each R and index calculate `Rl_i = R*l_i(0)`
1. Calculate `Rsg = sum(Rl_i)` - remember it
1. Remember `x_i`

### Phase 2:

Dealer:
1. Continuing with the selected peers subset from the previous phase - if subset differs anyhow, sign will be invalid
1. Send them the calculated `Rsg` and `Message`

Peer:
1. Regenerate `r = hash(Salt, hash(Message))`
1. Calculate `h = hash(Rsg, GroupPubKey, Message)`
1. Send `s = r + h*PrivateKey`

Dealer:
1. Collect `s_i` from `N` peers
1. Calculate the group sign as `gs = fit([(x_i, s_i) for x_i, s_i in zip(xes, collected_signs)])(0)`

In [4]:
# Signing

message = b"Test message for signing"
R_sg = generator * 0
xes: List[int] = []

# Phase 1: Each peer generates their partial signature
for peer in peers[:N]:
    i, R = peer.phase1(message)
    R_sg += R*lx(i, 0)
    xes.append(x_i(i))

# Phase 2: Each peer calculates their s_i
collected_signs: List[int] = []
for peer in peers[:N]:
    s_i = peer.phase2(R_sg, message)
    collected_signs.append(s_i)

# The final signature is (R_sg, gs)
gs = fit([(x_i, s_i) for x_i, s_i in zip(xes, collected_signs)], generator.order)(0)
signature = (R_sg, gs)

# TODO: wtf does not work

# Verification
def verify_signature(message: bytes, signature: Tuple[E, int], public_k: E):
    R, s = signature
    h = message_hash(R, public_k, message)
    return R + public_k*h == generator * s

assert verify_signature(message, signature, group_public_key)

(1711583739, 1934647547)


AssertionError: 

## Elaborations

The key operation is `gs = fit([(x_i, s_i) for x_i, s_i in zip(xes, collected_signs)])(0)` that effectively utilizes the principles of polynomial interpolation:

Consider a polynomial $f(x)$ that fits a set of points $(x_i, y_i)$. If another polynomial $g(x)$ is constructed such that each point is transformed linearly, where $\hat{y}_i = k \cdot y_i + c_i$, then $g(x)$ can be derived from $f(x)$ using the coefficients $k$ and $c_i$.

This relationship is described by the equation:

$$ g(x) = k \cdot f(x) + \sum c_i \cdot l_i(x) $$

Here, $l_i(x)$ represents the Lagrange basis polynomials. This formulation is valid even within the confines of modular arithmetic.

If $f(x)$ is defined by the private keys of a group and we are interested in a specific point, for instance $f(0)$ (as the representative of the group), the described method shows that setting $r = \sum c_i \cdot l_i(x)$ and $ k = h$ leads to the equation $ s = h \cdot f(0) + r$. This $ s$ corresponds to the valid signature for the "group" public key.

To generate such a signature, N peers must concur on the value of $r$. This consensus is maintained in such a way that the exact value of $r$ remains undisclosed; peers are only aware of its encrypted form. Despite multiple parties agreeing on its encrypted form, extracting $r$ from $s$ requires knowledge of the group's private key, and conversely, obtaining the group's private key explicitly requires $r$ in an unencrypted form.

### Attacks:

1. If hacker got access to N private keys - he can make just the single private key for the entire group